<a href="https://colab.research.google.com/github/HEKOCMOC/HPC_LAB/blob/main/Matrix_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime', 'float64']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [11]:
import math
import numpy as np
from numba import cuda, jit, float64, prange
from numba.cuda import jit 
import time
from datetime import datetime 

cuda.detect()

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

In [30]:
N = 2000
A = np.random.randn(N, N)
B = np.random.randn(N, N)
C = np.zeros((N, N))

In [31]:
def matmul_CPU(A, B, C):
  for i in range(C.shape[0]):
    for j in range(C.shape[1]):
      sum = 0
      for k in range(A.shape[1]):
        sum += A[i, k] * B[k, j]
      C[i, j] = sum
      return C
host_1 = C

%time _= matmul_CPU(A, B, C)
start = datetime.now()
matmul_CPU(A, B, C)
print('--------------------------------------')
print('Matmul on CPU:', datetime.now() - start)
print('--------------------------------------')


CPU times: user 1.84 ms, sys: 0 ns, total: 1.84 ms
Wall time: 1.88 ms
--------------------------------------
Matmul on CPU: 0:00:00.001982
--------------------------------------


In [32]:
@cuda.jit
def matmul_GPU(A, B, C):
  i, j = cuda.grid(2)
  if i < C.shape[0] and j < C.shape[1]:
    sum = 0
    for k in range(A.shape[1]):
      sum += A[i, k] * B[k, j]
    C[i, j] = sum

def host_naive(A, B, C):
  d_A = cuda.to_device(A)
  d_B = cuda.to_device(B)
  d_C = cuda.device_array(C.shape, np.float64)

  threadsperblock = (16, 16)
  blockspergrid_x = math.ceil(A.shape[0]/threadsperblock[0])
  blockspergrid_y = math.ceil(B.shape[1]/threadsperblock[1])
  blockspergrid = (blockspergrid_x, blockspergrid_y)

  matmul_GPU[blockspergrid, threadsperblock](d_A, d_B, d_C)
  return d_C.copy_to_host()

host_2 = C
%time _= host_naive(A, B, C)
start = datetime.now()
host_naive(A, B, C)
print('--------------------------------------')
print('Matmul on GPU:', datetime.now() - start)
print('--------------------------------------')



CPU times: user 1.34 s, sys: 8.82 ms, total: 1.35 s
Wall time: 1.35 s
--------------------------------------
Matmul on GPU: 0:00:00.946033
--------------------------------------


In [33]:
print('Проверка корректности перемножения матриц') 
print('Если матрицы равны, то будет True:', np.allclose(host_1, host_2))

Проверка корректности перемножения матриц
Если матрицы равны, то будет True: True
